Course Instructor: Bernd Neumayr, JKU

# UE04- SPARQL Updata and RDF Datasets

Complete the **8 tasks (1 point per task)** in the `3. SPARQL Update` sheet of `SemAI.jar` first and then transfer them to this notebook.

For each task include:
- A headline including the task number
- The task description 
- Your solution in executable form: your solutions for SemAI.jar will make use of the default grap. In this notebook you have to transform your solutions according to the workaround exemplified in V04_SPARQL_Update.ipynb
- After executing the update request, print a serizalization of the dataset in TriG format.  

**Task 9 (2 points)**  is to develop a nice visualization of RDF datasets using `visualize_graph_pyvis` from UE02 as a starting point. The requirements are as follows:
- Each named graph must be represented as an independent graph. This means, for example, that :Jane in :JanesGraph is a different node than :Jane in :BillsGraph. There are no edges between nodes in different graphs.
- It is not strictly necessary to draw a box around each named graph, as seen in the slides. The different named graphs should simply be visually distinguishable and not overlap.
- If not all nodes within a named graph are connected, make sure in the visualization that the named graph still forms a coherent visual unit in some way.

## Preperations

In [ ]:
!pip install -q rdflib

### Imports and functions
We are re-using the sparql_select function.

In [ ]:
# Imports
import pandas as pd
import rdflib
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace


# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]                  
    rows.append(row)                    #     add the dictionary (row) to the list 
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      


## Task 1

In [ ]:
ds = rdflib.Dataset()
ds.parse(format="trig", data="""
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix : <http://example.org/> .

""")

ds.update("""
insert data {
  graph :main {
    :G1 :author :Mary .
    :G2 :author :Peter .
  }
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .

:main {
    :G1 :author :Mary .

    :G2 :author :Peter .
}




## Task 2

In [ ]:
ds.update("""
insert data {
  graph :G1 {
    :Mary :knows :Peter, :John, :Mary
  }.
  graph :G2 {
    :Peter :knows :Mary.
    :John :knows :Mary.
  }
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .

:main {
    :G1 :author :Mary .

    :G2 :author :Peter .
}

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}




## Task 3

In [ ]:
ds.update("""
insert{
  graph :main {
    ?p :knownBy ?f .
  }
}
where {
  graph :G2 {
    ?f :knows ?p .
  }
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .

:main {
    :G1 :author :Mary .

    :G2 :author :Peter .

    :Mary :knownBy :John,
            :Peter .
}

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}




## Task 4

In [ ]:
ds.update("""
delete where {graph :main {?p :knownBy ?f}}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .

:main {
    :G1 :author :Mary .

    :G2 :author :Peter .
}

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}




## Task 5

In [ ]:
ds.update("""
insert {
  graph :main {
    ?g :knowsCount ?cnt.
  }
} 
where {
  select ?g (count(?f) as ?cnt)
  where {
    graph ?g{
      ?p :knows ?f
    }
  }group by ?g
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:main {
    :G1 :author :Mary ;
        :knowsCount 3 .

    :G2 :author :Peter ;
        :knowsCount 2 .
}

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}




## Task 6

In [ ]:
ds.update("""
insert{
  graph :main {
    :ds :graphCount ?cnt
  }
}
where{
  select (count(?g) as ?cnt)
  where {
    graph ?g {}
    FILTER(?g in (:G1, :G2, :main))
  }
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:main {
    :G1 :author :Mary ;
        :knowsCount 3 .

    :G2 :author :Peter ;
        :knowsCount 2 .

    :ds :graphCount 3 .
}

:G1 {
    :Mary :knows :John,
            :Mary,
            :Peter .
}

:G2 {
    :John :knows :Mary .

    :Peter :knows :Mary .
}




## Task 7

In [ ]:
ds.update("""
delete{ graph :main{?g ?p ?o}}
insert{graph ?g{
?g ?p ?o.
}}
where {
Select ?g ?p ?o
WHERE {
GRAPH ?g {}
GRAPH :main { ?g ?p ?o }
}
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:main {
    :ds :graphCount 4 .
}

:G1 {
    :G1 :author :Mary ;
        :knowsCount 3 .

    :Mary :knows :John,
            :Mary,
            :Peter .
}

:G2 {
    :G2 :author :Peter ;
        :knowsCount 2 .

    :John :knows :Mary .

    :Peter :knows :Mary .
}




## Task 8

In [ ]:
ds.update("""
DELETE { GRAPH ?g {
?g :knowsCount ?c
} }
INSERT { 
GRAPH ?g {
?aut :knows :Susi .
} 
}
WHERE {
Select ?g ?aut ?c
WHERE {
GRAPH ?g { ?g :author ?aut . ?g :knowsCount ?c}
}
};
INSERT { GRAPH ?g2 { ?g2 :knowsCount ?xcount } }
WHERE {
Select ?g2 (count(?x) as ?xcount)
WHERE {
GRAPH ?g2
{
?x :knows ?y
} }
GROUP BY ?g2
}
""")

print(ds.serialize(format="trig"))

@prefix : <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:main {
    :ds :graphCount 4 .
}

:G1 {
    :G1 :author :Mary ;
        :knowsCount 4 .

    :Mary :knows :John,
            :Mary,
            :Peter,
            :Susi .
}

:G2 {
    :G2 :author :Peter ;
        :knowsCount 3 .

    :John :knows :Mary .

    :Peter :knows :Mary,
            :Susi .
}


